In [1]:
import pandas as pd
import numpy as np
import keras
from keras import optimizers
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM
from sklearn.model_selection import GridSearchCV, TimeSeriesSplit
from sklearn.preprocessing import StandardScaler
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from numpy.random import seed
seed(69)

Using TensorFlow backend.


In [2]:
# Importing the data
data = pd.read_csv('table3.csv')
data.insert(loc = 1, column = 'F_Returns', value = data['M_Returns'].shift(-1))
data = data.dropna()

In [3]:
# Splitting the data into training, validation and test set
TrainingSet = data.iloc[0: round(0.4*data.shape[0]), :]

ValidationSet = data.iloc[round(0.4*data.shape[0]): round(0.8*data.shape[0]), :]

OOSTraining = data.iloc[0: round(0.8*data.shape[0]), :]

TestSet = data.iloc[round(0.8*data.shape[0]):, :]

In [4]:
# Splitting the data into Training, validation and test set
X_train = TrainingSet.drop(['F_Returns'], axis=1).values
y_train = TrainingSet['F_Returns'].values

X_validation = ValidationSet.drop(['F_Returns'], axis=1).values
y_validation = ValidationSet['F_Returns'].values

X_oostrain = OOSTraining.drop(['F_Returns'], axis=1).values
y_oostrain = OOSTraining['F_Returns'].values

X_test = TestSet.drop(['F_Returns'], axis=1).values
y_test = TestSet['F_Returns'].values

In [5]:
# Scaling the data
sc_xtr = StandardScaler()
sc_xv = StandardScaler()
sc_xoostr = StandardScaler()
sc_xte = StandardScaler()
X_train = sc_xtr.fit_transform(X_train)
X_validation = sc_xv.fit_transform(X_validation)
X_oostrain = sc_xoostr.fit_transform(X_oostrain)
X_test = sc_xte.fit_transform(X_test)

In [6]:
y_train

array([ 0.040893, -0.001664,  0.091667, -0.015573, -0.053125, -0.049505,
        0.043056,  0.15436 , -0.10465 , -0.032468,  0.10201 , -0.018405,
        0.0225  ,  0.089506,  0.10482 , -0.086667, -0.070822,  0.057927,
       -0.091643,  0.012821, -0.012658, -0.15    , -0.007634,  0.1     ,
        0.007692,  0.17193 ,  0.065868, -0.10618 ,  0.079365,  0.1     ,
       -0.039572,  0.067416,  0.11053 ,  0.069194, -0.089286, -0.071078,
        0.042744,  0.015306,  0.25126 ,  0.00241 , -0.064516, -0.092672,
        0.055107,  0.088435,  0.008333,  0.064463, -0.09375 ,  0.017241,
       -0.061017,  0.004545,  0.076923, -0.07521 ,  0.006865, -0.054545,
        0.094231, -0.042035,  0.11316 ,  0.017012, -0.034908, -0.080851,
        0.049074,  0.033333, -0.10538 , -0.059615,  0.010309,  0.15306 ,
        0.035841,  0.030108, -0.014614,  0.066102,  0.076   ,  0.11524 ,
        0.028667,  0.039088,  0.079937, -0.014224,  0.034024,  0.005722,
        0.062873,  0.004032, -0.02008 , -0.012022, 

In [7]:
# Creating a function that calculates the out of sample R^2
def my_R_oos_sq(y_test, y_pred):
    Er_squared = (y_test - y_pred)**2

    SE = 0
    for i in range(len(Er_squared)):
        SE = SE + (float(Er_squared[i]))
    
    SR = 0
    for j in range(len(y_test)):
        SR = SR + (float(y_test[j]))**2
    
    R_oos_sq = 1 - (SE/SR)
    return R_oos_sq

### Deep Neural Nets

In [8]:
# Hyperparameters to be optimzed

lr = [0.1, 0.01, 0.001]
optimizer = ['adam', 'SGD', 'RMSprop']
kernel_initializer = ['normal', 'uniform']
batch_size = [15, 30, 60]
epochs = [100, 150, 200]
#activation = ['relu', 'elu', 'tanh']

In [9]:
def build_regressor(optimizer = optimizer, lr = lr, kernel_initializer = kernel_initializer):
    # Initialising the ANN
    regressor = Sequential()
    # Adding the input layer and the first hidden layer
    regressor.add(Dense(units = 32, kernel_initializer = kernel_initializer, activation = 'relu', input_dim = 6))
    # Adding dropout for NN training...Should comment out in out of sample
    regressor.add(Dropout(0.5))
    # Adding the second hidden layer
    regressor.add(Dense(units = 16, kernel_initializer = kernel_initializer, activation = 'relu'))
    # Adding dropout for NN training...Should comment out in out of sample
    regressor.add(Dropout(0.5))
    # Adding the third hidden layer
    regressor.add(Dense(units = 8, kernel_initializer = kernel_initializer, activation = 'relu'))
    # Adding the output layer
    regressor.add(Dense(units = 1, kernel_initializer = kernel_initializer, activation = 'softsign'))
    # Compiling the ANN
    regressor.compile(optimizer = optimizer, loss = 'mean_squared_error')
    return regressor

In [9]:
regressor = KerasRegressor(build_fn = build_regressor)

In [13]:
#regressor.fit(X_oostrain, y_oostrain, batch_size = 30, epochs = 150)
#y_pred = regressor.predict(X_test)

In [11]:
# Grid Search for hyperparameters optimization

param_grid = dict(optimizer = optimizer, kernel_initializer = kernel_initializer, batch_size = batch_size, epochs = epochs)

tscv = TimeSeriesSplit(n_splits = 5)
grid_search = GridSearchCV(estimator = regressor, param_grid = param_grid, scoring = 'r2', cv = tscv, n_jobs = -1)
grid_search = grid_search.fit(X = X_train, y = y_train)

# summarize results
print("Best: %f using %s" % (grid_search.best_score_, grid_search.best_params_))
means = grid_search.cv_results_['mean_test_score']
stds = grid_search.cv_results_['std_test_score']
params = grid_search.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param)) 

/opt/anaconda3/lib/python3.7/site-packages/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning


Epoch 1/150
259/259 [==============================] - 0s 2ms/step - loss: 0.0072
Epoch 2/150
259/259 [==============================] - 0s 198us/step - loss: 0.0072
Epoch 3/150
259/259 [==============================] - 0s 180us/step - loss: 0.0073
Epoch 4/150
259/259 [==============================] - 0s 178us/step - loss: 0.0070
Epoch 5/150
259/259 [==============================] - 0s 181us/step - loss: 0.0069
Epoch 6/150
259/259 [==============================] - 0s 176us/step - loss: 0.0069
Epoch 7/150
259/259 [==============================] - 0s 180us/step - loss: 0.0069
Epoch 8/150
259/259 [==============================] - 0s 173us/step - loss: 0.0067
Epoch 9/150
259/259 [==============================] - 0s 175us/step - loss: 0.0069
Epoch 10/150
259/259 [==============================] - 0s 174us/step - loss: 0.0068
Epoch 11/150
259/259 [==============================] - 0s 177us/step - loss: 0.0069
Epoch 12/150
259/259 [==============================] - 0s 172us/step - loss

In [11]:
# Testing the R Squared out of sample
my_R_oos_sq(y_test, y_pred)

0.04020058046171093

In [14]:
# Output Binary Classification
Pred_NN = []
for i in range(len(y_pred)):
    if y_pred[i] >= 0:
        Pred_NN.append(1)
    else:
        Pred_NN.append(0)
        
Val = []
for j in range(len(y_test)):
    if y_test[j] >= 0:
        Val.append(1)
    else:
        Val.append(0)
        
count = 0
for k in range(len(y_pred)):
    if Pred_NN[k] == Val[k]:
        count += 1
res_NN = count/len(y_pred)
res_NN

NameError: name 'y_pred' is not defined

### LSTM

In [7]:
X_oostrain = np.reshape(X_oostrain, (X_oostrain.shape[0], 1, X_oostrain.shape[1]))
X_test = np.reshape(X_test, (X_test.shape[0], 1, X_test.shape[1]))

In [8]:
X_oostrain.shape

(518, 1, 6)

In [9]:
def build_LSTMregressor():
    LSTMregressor = Sequential()
    LSTMregressor.add(LSTM(units = 60, input_shape = (1, 6)))
    LSTMregressor.add(Dense(units = 1, activation = 'softsign'))
    LSTMregressor.compile(optimizer = 'adam', loss = 'mean_squared_error')
    return LSTMregressor

In [10]:
LSTMregressor = KerasRegressor(build_fn = build_LSTMregressor)

In [11]:
LSTMregressor.fit(X_oostrain, y_oostrain, batch_size = 30, epochs = 100)
y_pred_LSTM = LSTMregressor.predict(X_test)

Epoch 1/100
518/518 [==============================] - 1s 2ms/step - loss: 0.0101
Epoch 2/100
518/518 [==============================] - 0s 168us/step - loss: 0.0071
Epoch 3/100
518/518 [==============================] - 0s 173us/step - loss: 0.0069
Epoch 4/100
518/518 [==============================] - 0s 184us/step - loss: 0.0068
Epoch 5/100
518/518 [==============================] - 0s 173us/step - loss: 0.0067
Epoch 6/100
518/518 [==============================] - 0s 168us/step - loss: 0.0068
Epoch 7/100
518/518 [==============================] - 0s 165us/step - loss: 0.0067
Epoch 8/100
518/518 [==============================] - 0s 165us/step - loss: 0.0067
Epoch 9/100
518/518 [==============================] - 0s 167us/step - loss: 0.0067
Epoch 10/100
518/518 [==============================] - 0s 165us/step - loss: 0.0067
Epoch 11/100
518/518 [==============================] - 0s 166us/step - loss: 0.0067
Epoch 12/100
518/518 [==============================] - 0s 165us/step - loss

In [12]:
# Testing the R Squared out of sample
my_R_oos_sq(y_test, y_pred_LSTM)

0.07998170363518631

In [13]:
# Output Binary Classification
Pred_LSTM = []
for i in range(len(y_pred_LSTM)):
    if y_pred_LSTM[i] >= 0:
        Pred_LSTM.append(1)
    else:
        Pred_LSTM.append(0)
    
Val = []
for j in range(len(y_test)):
    if y_test[j] >= 0:
        Val.append(1)
    else:
        Val.append(0)
    
count = 0
for k in range(len(y_pred_LSTM)):
    if Pred_LSTM[k] == Val[k]:
        count += 1
res_LSTM = count/len(y_pred_LSTM)
res_LSTM

0.5038759689922481

### Random Forest

In [10]:
clf = RandomForestRegressor(n_estimators = 1000, criterion = 'mse', 
                            min_samples_split = 40, min_samples_leaf = 10, random_state = 69)

In [11]:
clf.fit(X_oostrain, y_oostrain)
y_pred_r = clf.predict(X_test)

In [21]:
# Grid Search for hyperparameters optimization
criterion = ['mse', 'mae']
n_estimators = [500, 1000]
min_samples_split = [20, 25, 30, 35, 40]
min_samples_leaf = [1, 3, 5, 7, 10]

param_grid = dict(min_samples_split = min_samples_split, n_estimators = n_estimators, 
                  criterion = criterion, min_samples_leaf = min_samples_leaf)

tscv = TimeSeriesSplit(n_splits = 5)
grid_search = GridSearchCV(estimator = clf, param_grid = param_grid, scoring = 'r2', cv = tscv, n_jobs = -1)
grid_search = grid_search.fit(X = X_train, y = y_train)

# summarize results
print("Best: %f using %s" % (grid_search.best_score_, grid_search.best_params_))
means = grid_search.cv_results_['mean_test_score']
stds = grid_search.cv_results_['std_test_score']
params = grid_search.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param)) 

Best: -0.058383 using {'criterion': 'mse', 'min_samples_leaf': 10, 'min_samples_split': 40, 'n_estimators': 1000}
-0.206955 (0.131810) with: {'criterion': 'mse', 'min_samples_leaf': 1, 'min_samples_split': 20, 'n_estimators': 500}
-0.198339 (0.122959) with: {'criterion': 'mse', 'min_samples_leaf': 1, 'min_samples_split': 20, 'n_estimators': 1000}
-0.182466 (0.126269) with: {'criterion': 'mse', 'min_samples_leaf': 1, 'min_samples_split': 25, 'n_estimators': 500}
-0.175007 (0.118510) with: {'criterion': 'mse', 'min_samples_leaf': 1, 'min_samples_split': 25, 'n_estimators': 1000}
-0.147912 (0.134094) with: {'criterion': 'mse', 'min_samples_leaf': 1, 'min_samples_split': 30, 'n_estimators': 500}
-0.142500 (0.126059) with: {'criterion': 'mse', 'min_samples_leaf': 1, 'min_samples_split': 30, 'n_estimators': 1000}
-0.132541 (0.126506) with: {'criterion': 'mse', 'min_samples_leaf': 1, 'min_samples_split': 35, 'n_estimators': 500}
-0.127784 (0.121121) with: {'criterion': 'mse', 'min_samples_lea

In [12]:
# Testing the R Squared out of sample
my_R_oos_sq(y_test, y_pred_r)

0.064354672908006

In [16]:
y_pred_r

array([ 2.83694301e-04,  1.16756612e-03, -1.17040101e-02,  5.26373607e-03,
        4.86441558e-03, -3.96670457e-03,  2.84780091e-03, -6.65735748e-03,
        5.97775513e-02,  2.97080357e-02,  1.72793915e-02,  2.50137011e-02,
        6.99564640e-02,  7.10755690e-02,  3.47872834e-02,  3.99780218e-02,
        2.83156698e-02,  2.22104506e-02,  8.38719716e-03,  6.03479782e-03,
        6.38502240e-03,  6.53010048e-03,  2.00845832e-03,  3.86180998e-02,
        3.87815503e-02,  3.37982702e-02,  4.56772926e-03,  4.79829316e-02,
        1.44048593e-02,  3.66188044e-03,  2.45103722e-03,  1.82521529e-03,
        3.73013140e-03,  3.26270924e-03, -3.24591073e-03,  4.08627492e-03,
        6.47008694e-03,  6.10859611e-03, -6.51070637e-04,  1.16704789e-03,
        1.35046906e-02, -7.15914036e-03, -4.28587934e-03,  4.69726883e-02,
        2.48505528e-02,  2.56902612e-02,  2.88344257e-02,  1.35193436e-02,
        1.19831382e-03,  3.43856670e-03, -6.95993230e-03,  6.05816561e-03,
        5.74647348e-03,  

In [15]:
# Output Binary Classification
Pred_RF = []
for i in range(len(y_pred_r)):
    if y_pred_r[i] >= 0:
        Pred_RF.append(1)
    else:
        Pred_RF.append(0)
        
count = 0
for k in range(len(y_pred_r)):
    if Pred_RF[k] == Val[k]:
        count += 1
res_RF = count/len(y_pred)
res_RF

NameError: name 'Val' is not defined

### XG Boost

In [16]:
cla = XGBRegressor(max_depth = 4, learning_rate = 0.1, n_estimators = 100, verbosity = 1,
    silent = None, booster = 'gbtree', objective = 'reg:squarederror', n_jobs = 1, nthread = None, 
    gamma = 0, min_child_weight = 1, max_delta_step = 0, subsample = 1, colsample_bytree = 1, 
    colsample_bylevel = 1, colsample_bynode = 1, reg_alpha = 0, reg_lambda = 1, scale_pos_weight = 1, 
    base_score = 0.5, random_state= 69, seed = None, missing = None, importance_type = 'gain')

In [17]:
cla.fit(X_oostrain, y_oostrain)
y_pred_boost = cla.predict(X_test)

In [18]:
# Testing the R Squared out of sample
my_R_oos_sq(y_test, y_pred_boost)

0.12708786512701287

In [19]:
# Output Binary Classification
Pred_XGB = []
for i in range(len(y_pred_boost)):
    if y_pred_boost[i] >= 0:
        Pred_XGB.append(1)
    else:
        Pred_XGB.append(0)
        
count = 0
for k in range(len(y_pred)):
    if Pred_XGB[k] == Val[k]:
        count += 1
res_XGB = count/len(y_pred)
res_XGB

0.5503875968992248

### Ensemble Method 

In [20]:
def ensemble(a, b, c):
    Pred_ENS = []
    for i in range(len(a)):
        if a[i] + b[i] + c[i] == 3:
            Pred_ENS.append(1)
        elif a[i] + b[i] + c[i] == 2:
            Pred_ENS.append(1)
        elif a[i] + b[i] + c[i] == 1:
            Pred_ENS.append(0)
        elif a[i] + b[i] + c[i] == 0:
            Pred_ENS.append(0)
    return Pred_ENS

In [21]:
Pred_ENS = ensemble(Pred_NN, Pred_RF, Pred_XGB)

count = 0
for k in range(len(y_pred)):
    if Pred_ENS[k] == Val[k]:
        count += 1
res_ENS = count/len(y_pred)
res_ENS

0.5736434108527132